In [183]:
import ee
import geemap
import sys

try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print(e)
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise
    
def add_metrics(image):
    ndvi = image.normalizedDifference(['B7', 'B4']).rename('NDVI')
    gndvi = image.normalizedDifference(['B7', 'B3']).rename('GNDVI')   
    savi = ee.Image().expression('((NIR-Red)/(NIR+Red+0.5))*1.5', {
        'NIR': image.select('B8').multiply(0.0001),
        'Red': image.select('B4').multiply(0.0001)
      }).rename('SAVI')
#     cvi = ee.Image().expression('(NIR/Green)*(Red/Green)', {
#         'NIR': input.select('B8').multiply(0.0001),
#         'Red': input.select('B4').multiply(0.0001),
#         'Green': input.select('B3').multiply(0.0001)
#       }).rename('SAVI')
#     evi = image.expression(
#       '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
#       'NIR' : image.select('B8').divide(10000),
#       'RED' : image.select('B4').divide(10000),
#       'BLUE': image.select('B2').divide(10000)}).rename('EVI')
    return image.addBands([ndvi, gndvi, savi])




Google Earth Engine has initialized successfully!


In [166]:
Map1 = geemap.Map(location=[52.244439, 6.843240], zoom=15)
Map1.add_basemap('SATELLITE')
Map1


Map(center=[52.244439, 6.84324], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

In [167]:
fc = ee.FeatureCollection(Map1.draw_last_feature)
region = fc.geometry()
cords = region.getInfo()['coordinates']
cords

[[[6.820716, 52.144146],
  [6.908266, 52.144146],
  [6.908266, 52.181317],
  [6.820716, 52.181317],
  [6.820716, 52.144146]]]

In [95]:
region = ee.Geometry.Polygon([[[6.758719, 52.230826],
  [6.751321, 52.225126],
  [6.753724, 52.221972],
  [6.750463, 52.217239],
  [6.750652, 52.214947],
  [6.751253, 52.211792],
  [6.745725, 52.209778],
  [6.745827, 52.20104],
  [6.761105, 52.20104],
  [6.770805, 52.195042],
  [6.773208, 52.1952],
  [6.778358, 52.20246],
  [6.764029, 52.212382],
  [6.759578, 52.216657],
  [6.757818, 52.218444],
  [6.755501, 52.218234],
  [6.76138, 52.221941],
  [6.765286, 52.224702],
  [6.76859, 52.222835],
  [6.769577, 52.223545],
  [6.765801, 52.226752],
  [6.763312, 52.225043],
  [6.761809, 52.225858],
  [6.759707, 52.227567],
  [6.761252, 52.228697],
  [6.758719, 52.230826]]])

In [168]:
myCollection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2018-06-01', '2018-08-31') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
listOfImages = myCollection.aggregate_array('system:index').getInfo()
print('Number of images in the collection: ', listOfImages)


Number of images in the collection:  ['20180607T104021_20180607T104022_T31UGT', '20180607T104021_20180607T104022_T32ULC', '20180702T104019_20180702T104021_T31UGT', '20180702T104019_20180702T104021_T32ULC', '20180707T104021_20180707T104515_T31UGT', '20180707T104021_20180707T104515_T32ULC', '20180712T104019_20180712T104020_T31UGT', '20180717T104021_20180717T104258_T31UGT', '20180717T104021_20180717T104258_T32ULC', '20180727T104021_20180727T104023_T31UGT', '20180727T104021_20180727T104023_T32ULC', '20180806T104021_20180806T104340_T31UGT', '20180806T104021_20180806T104340_T32ULC']


In [190]:
#%%
vis_params = {"min": 0, 
              "max": 1500, 
              "bands": ["B4", "B3", "B2"]}
Images = myCollection.toList(myCollection.size())

ndviParams = {min: 0, max: 1, "palette":  ['FF0000', '000000', '00FF00'], "bands": ['NDVI']};

gndviParams = {min: 0, max: 1, "palette":  ['FF0000', '000000', '00FF00'], "bands": ['GNDVI']};

saviParams = {min: 0, max: 1, "palette":  ['FF0000', '000000', '00FF00'], "bands": ['SAVI']};



In [188]:
Image_1 = ee.Image(Images.get(1)).clip(region)
Image_1_metrics = add_metrics(Image_1)

Image_2 = ee.Image(Images.get(3)).clip(region)
Image_2_metrics = add_metrics(Image_2)

Image_diff = (Image_1_metrics.subtract(Image_2_metrics)).abs()

Map3 = geemap.Map(location=[ 52.1952, 6.773208], zoom=15)
Map3.addLayer(Image_diff, ndviParams, "MedianImage", True) 
Map3




Map(center=[52.1952, 6.773208], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

In [196]:
Map2 = geemap.Map(location=[ 52.1952, 6.773208], zoom=15)
# Map1.add_basemap('SATELLITE')
Map2
Map2.addLayer(Image_diff, gndviParams, "MedianImage", True) 
Map2



Map(center=[52.1952, 6.773208], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

In [195]:
out_dir = '/Users/hungnguyen/Masters/DataScience/DSAssignments/GIS-project/KasUnBrot2/'
info = myCollection.aggregate_array('system:index').getInfo()
filename = out_dir + 'diff-07Jun-02Jul-2018' + '.tif'
Image_diff = Image_diff.select(["NDVI", "GNDVI", "SAVI"])
geemap.ee_export_image(
Image_diff, filename = filename, scale = 30, region = region, file_per_band = False)


Generating URL ...
Please wait ...
An error occurred while downloading.


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [192]:
Map4 = geemap.Map(location=[ 52.1952, 6.773208], zoom=15)
# Map1.add_basemap('SATELLITE')
Map4
Map4.addLayer(Image_diff, saviParams, "MedianImage", True) 
Map4



Map(center=[52.1952, 6.773208], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.